In [42]:
from flask import Flask,render_template, request
app = Flask(__name__)


'''
"What is identity? It is the difference between us. Difference is experienced in the mind, yet the Buddha said this mind creates the world, that this world only exists in the mind and nowhere else."
'''

from hashlib import sha256
import os
from binascii import * 
#from flag import flag
flag = "help"

class MerkleTree(object): 
    def __init__(self):
        self.leaves = list()
        self.levels = None
        self.is_ready = False

    def add_leaf(self, value):         
        self.leaves.append(sha256(value).digest())

    def _calculate_next_level(self):
        solo_leave = None
        N = len(self.levels[0])  # number of leaves on the level
        if N % 2 == 1:  # if odd number of leaves on the level
            solo_leave = self.levels[0][-1]
            N -= 1

        new_level = []
        for l, r in zip(self.levels[0][0:N:2], self.levels[0][1:N:2]):
            new_level.append(sha256(l+r).digest())
            print(sha256(l+r).digest())
            print((l+r).hex())
        if solo_leave is not None:
            new_level.append(solo_leave)
        self.levels = [new_level, ] + self.levels  # prepend new level

    def make_tree(self):
        self.is_ready = False
        if len(self.leaves) > 0:
            self.levels = [self.leaves, ]
            while len(self.levels[0]) > 1:
                self._calculate_next_level()
        self.is_ready = True

    def get_merkle_root(self):
        if self.is_ready:
            if self.levels is not None:
                return self.levels[0][0]
            else:
                return None
        else:
            return None

def hashPassword(p):
    mt = MerkleTree()
    for c in p: 
        mt.add_leaf(c)
    mt.make_tree()
    return hexlify(mt.get_merkle_root())

def generateRandomPassword(length):
    p = os.urandom(length)
    return hexlify(p)

def splitPassword(p, isString=True):
    if isString:
        return ",".join([c for c in p]).split(",")
    else:
        return b",".join([chr(c).encode() for c in p]).split(b",")

def validatePassword(user_password, password_hashes, denyList=[], debug=False):    
    try:
        joined_password = unhexlify("".join(user_password.split(",")))
    except Exception as e: 
        raise Exception("ERROR: Formatting error. Exiting")
        
    if joined_password in denyList: 
        raise Exception("Nice try, but that password is not allowed...")
    
    split_password = [unhexlify(c) for c in user_password.split(",")]
    user_password_hash = hashPassword(split_password)

    if debug: 
        print("user_password entered: [", user_password, "]")
        print("hashes", password_hashes)
        print("deny list", denyList)
        print("hash", user_password_hash)

    if (user_password_hash in password_hashes): 
        return True

    return False

test_password = b"SuperSecretPassword"
test_password_list = splitPassword(test_password, isString=False)
test_password_hash = hashPassword(test_password_list)
password_hashes = [test_password_hash]

for i in range(0,9):
    new_password = generateRandomPassword(32)
    new_password_list = splitPassword(new_password, isString=False)
    new_password_hash = hashPassword(new_password_list)
    password_hashes.append(new_password_hash)

@app.route('/')
def hello_world():
    return render_template("login.html")

@app.route('/auth', methods=['GET'])
def do_auth():
    TP = request.args.get('transformed','')
    P = request.args.get('password','').encode()
    
    if P == test_password:
        return 'Nice try, but that password is not allowed :P'

    try:
        res = validatePassword(TP, password_hashes, denyList=[test_password])
    except Exception as e: 
        return str(e)    

    if res:
        return 'Authed! Here is your flag: '+flag
    else:
        return 'Wrong Password'

b'w\xb3\xdeG\x01\xea,)O\xe5\x92e\xf6cb\xe4\x87&\xabi\x0c\x0b\xf9O\x02u\x13\xd4\x02\xe8\x93\xbe'
8de0b3c47f112c59745f717a626932264c422a7563954872e237b223af4ad6430bfe935e70c321c7ca3afc75ce0d0ca2f98b5422e008bb31c00c6d7f1f1c0ad6
b'\xb5\x97WHbJ\x8b\xe4d\xf0$\xf3\x9a\xdc\xb0\xae\xadc?6\xb2\x8b(\xa0\xf6M\xb4xJK\x1c!'
148de9c5a7a44d19e56cd9ae1a554bf67847afb0c58f6e12fa29ac7ddfca99403f79bb7b435b05321651daefd374cdc681dc06faa65e374e38337b88ca046dea
b'\x8e\x01p\xc1\rS\xd5b\xe5!f\xc44\xaf\x11\x9aM\xd5\xc1\x0f\xff\x0b\x8c\xec\xa1\x99\x8e\xa6\xf4\xf2\xec\xb2'
454349e422f05297191ead13e21d3db520e5abef52055e4964b82fb213f593a18de0b3c47f112c59745f717a626932264c422a7563954872e237b223af4ad643
b'k\xfeA~\xf7\x03$\xf1\xf88\x8d\xf7 \x07\x81\xab\x1d\x07\x84e\x15{\xce\x97\xee\xb5n\xd6\x0b\x87v\xb6'
3f79bb7b435b05321651daefd374cdc681dc06faa65e374e38337b88ca046dea2e7d2c03a9507ae265ecf5b5356885a53393a2029d241394997265a1a25aefc6
b'g\xa36\x8bUT\xdd\x84Z\xacj\xac]\xc138D\x93\x8fS\xdf>\x95i\x07\x96\xb8\xbc\xad\xb3\xc7\x1

In [2]:
password_hashes[0]

b'af0bd175f2c542c6aacf667c5c270a4c13a8d1cb6895807d6a9ed5d74fe71307'

In [18]:
a = generateRandomPassword(32); print(a)

b'cd6945c693e715197e2f12c464a96ffcb7416d8026041ad77ce82edb76cc870d'


In [19]:
hashPassword(splitPassword(a, isString=False))

b'abebc476df7796929775ffee58478decd0dff2d4cf99e4f596a3aaacfc1c6cea'

In [43]:
aa = MerkleTree()
for c in test_password_list:
    aa.add_leaf(c)
aa.make_tree()

b'w\xb3\xdeG\x01\xea,)O\xe5\x92e\xf6cb\xe4\x87&\xabi\x0c\x0b\xf9O\x02u\x13\xd4\x02\xe8\x93\xbe'
8de0b3c47f112c59745f717a626932264c422a7563954872e237b223af4ad6430bfe935e70c321c7ca3afc75ce0d0ca2f98b5422e008bb31c00c6d7f1f1c0ad6
b'\xb5\x97WHbJ\x8b\xe4d\xf0$\xf3\x9a\xdc\xb0\xae\xadc?6\xb2\x8b(\xa0\xf6M\xb4xJK\x1c!'
148de9c5a7a44d19e56cd9ae1a554bf67847afb0c58f6e12fa29ac7ddfca99403f79bb7b435b05321651daefd374cdc681dc06faa65e374e38337b88ca046dea
b'\x8e\x01p\xc1\rS\xd5b\xe5!f\xc44\xaf\x11\x9aM\xd5\xc1\x0f\xff\x0b\x8c\xec\xa1\x99\x8e\xa6\xf4\xf2\xec\xb2'
454349e422f05297191ead13e21d3db520e5abef52055e4964b82fb213f593a18de0b3c47f112c59745f717a626932264c422a7563954872e237b223af4ad643
b'k\xfeA~\xf7\x03$\xf1\xf88\x8d\xf7 \x07\x81\xab\x1d\x07\x84e\x15{\xce\x97\xee\xb5n\xd6\x0b\x87v\xb6'
3f79bb7b435b05321651daefd374cdc681dc06faa65e374e38337b88ca046dea2e7d2c03a9507ae265ecf5b5356885a53393a2029d241394997265a1a25aefc6
b'g\xa36\x8bUT\xdd\x84Z\xacj\xac]\xc138D\x93\x8fS\xdf>\x95i\x07\x96\xb8\xbc\xad\xb3\xc7\x1

In [45]:
payload = "55d7c617af58ca7e9dcb9e27e154b821f0197d945affa5def4fa13ce4564db4cff2932604840b9c31a8f6002fe28f45d8955aa88035e370471fb2b02f087b71d" + "," + "36068c29db774dd798c87e1b9f71de62e7f6a2557afec8395c55ae3259cedbaa18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4"
print(payload)

55d7c617af58ca7e9dcb9e27e154b821f0197d945affa5def4fa13ce4564db4cff2932604840b9c31a8f6002fe28f45d8955aa88035e370471fb2b02f087b71d,36068c29db774dd798c87e1b9f71de62e7f6a2557afec8395c55ae3259cedbaa18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4


In [46]:
validatePassword(payload, password_hashes, denyList=[test_password], debug=True)

b"\xaf\x0b\xd1u\xf2\xc5B\xc6\xaa\xcff|\\'\nL\x13\xa8\xd1\xcbh\x95\x80}j\x9e\xd5\xd7O\xe7\x13\x07"
606755aeed86f9892329e3dc622bb285910dea2cab41da929217bb75e569a1ce42343d1bc3021c2e7cd8bcb26928e3dc42be0c3853849888518e888414789a9e
user_password entered: [ 55d7c617af58ca7e9dcb9e27e154b821f0197d945affa5def4fa13ce4564db4cff2932604840b9c31a8f6002fe28f45d8955aa88035e370471fb2b02f087b71d,36068c29db774dd798c87e1b9f71de62e7f6a2557afec8395c55ae3259cedbaa18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4 ]
hashes [b'af0bd175f2c542c6aacf667c5c270a4c13a8d1cb6895807d6a9ed5d74fe71307', b'53597203ee9ca94dcd59c914c4f814543119d71443ac25824d583f9d8d0c9d42', b'0a56d36d10df205f7b3197d0d2d4f3797af8511b87f54fd8df7dcf0e965ff9a6', b'de7c1eaa1ff7dd029cf11c917934c621f8487c58d06581d71fe255d0e667b665', b'c5ee8f3e9f8d4f4c867be05ed446771cf5a8ccce40b69e372e33e3303c213a5b', b'ade026b0b1e1925a551ca33055ea98196a89f9fab1368ecbb59b7cbdbb8aa64c', b'10307dbd7df97e2bc536e649deba9ba8e9a848b37f8e85663670e7722fe29ad7

True

In [38]:
aa.levels[1]

[b'`gU\xae\xed\x86\xf9\x89#)\xe3\xdcb+\xb2\x85\x91\r\xea,\xabA\xda\x92\x92\x17\xbbu\xe5i\xa1\xce',
 b'B4=\x1b\xc3\x02\x1c.|\xd8\xbc\xb2i(\xe3\xdcB\xbe\x0c8S\x84\x98\x88Q\x8e\x88\x84\x14x\x9a\x9e']

In [31]:
aa.levels[1][0].hex()+','+aa.levels[1][1].hex()

'606755aeed86f9892329e3dc622bb285910dea2cab41da929217bb75e569a1ce,42343d1bc3021c2e7cd8bcb26928e3dc42be0c3853849888518e888414789a9e'

In [32]:
payload = aa.levels[1][0].hex()+','+aa.levels[1][1].hex()

In [35]:
validatePassword(payload, password_hashes, denyList=[test_password], debug=True)

user_password entered: [ 606755aeed86f9892329e3dc622bb285910dea2cab41da929217bb75e569a1ce,42343d1bc3021c2e7cd8bcb26928e3dc42be0c3853849888518e888414789a9e ]
hashes [b'af0bd175f2c542c6aacf667c5c270a4c13a8d1cb6895807d6a9ed5d74fe71307', b'ffa3469ad6da3debcc00487910370a012574c2048af7451049c19f8766939e03', b'947cebedac42e8f992ff3bbf5dc0995d80538a5fcb7046bc912ea2ece7f5c7f7', b'7816bedf4da46c75811674f40e855fb978cf4c6a81a45cdfd40c0b3111025560', b'7f495c1572a603e7f5645d7b283b67e46ccede96eb25ef6c311a4990406b27b3', b'655938de459f028a1ea10459d85bf385cdbe9c7cd5bdb212e66e80b7aaab963e', b'4ad32423173bb0265d00e72435172be8d68c27fc23d1b9ba677dd3a345e6b2ec', b'59d900d8d641b3bfbbd57049b858e4be121ab070f4de77e1b48e669dea59d4fe', b'9d9bb7978b42b335d2b9fc91e8594cf2df99870ed74c55847814aa9c4af7b4e1', b'166fa6ade3b96b0e9370ff180f47ad2176221d1d7e685c920bf8953e462ba852']
deny list [b'SuperSecretPassword']
hash b'8dffb9a4bd415199403f2a9813b88bc5f0af4a8b64ac808312dd19d0dc5d2475'


False

In [34]:
payload

'606755aeed86f9892329e3dc622bb285910dea2cab41da929217bb75e569a1ce,42343d1bc3021c2e7cd8bcb26928e3dc42be0c3853849888518e888414789a9e'

In [36]:
split_password = [unhexlify(c) for c in payload.split(",")]
user_password_hash = hashPassword(split_password)

In [37]:
split_password

[b'`gU\xae\xed\x86\xf9\x89#)\xe3\xdcb+\xb2\x85\x91\r\xea,\xabA\xda\x92\x92\x17\xbbu\xe5i\xa1\xce',
 b'B4=\x1b\xc3\x02\x1c.|\xd8\xbc\xb2i(\xe3\xdcB\xbe\x0c8S\x84\x98\x88Q\x8e\x88\x84\x14x\x9a\x9e']

In [39]:
hashPassword(split_password)

b'8dffb9a4bd415199403f2a9813b88bc5f0af4a8b64ac808312dd19d0dc5d2475'

In [20]:
while True:
    
    a = generateRandomPassword(32)
    
    b = hashPassword(splitPassword(a, isString=False))
    
    if b == password_hashes[0]:
        
        print(a)
        print(b)
        break

KeyboardInterrupt: 